In [1]:
# Import libraries

import cv2
import numpy as np
import matplotlib.pyplot as plt


In [9]:
# Define function for cell segmentation

def segment_cells(image):
    # Preprocessing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresholded = cv2.threshold(blurred, 120, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Morphological operations to clean up the image
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    opening = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Finding sure foreground area using distance transform
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    # Marker labelling
    _, markers = cv2.connectedComponents(sure_fg)

    # Add 1 to all markers to differentiate background and cells
    markers = markers + 1

    # Mark the unknown region as 0
    markers[unknown == 255] = 0

    # Apply watershed algorithm
    markers = cv2.watershed(image, markers)
    image[markers == -1] = [0, 0, 255]

    # Return the segmented image
    return image


In [11]:
def segment_image(image):
    """
    Segment roundish cell objects in the image
    """
    original = image.copy()


    if len(image.shape) == 3:  # Check if the image is color (3 channels)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:  # Grayscale image
        gray = image.copy()


    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Adaptive thresholding to obtain binary image
    _, threshold = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Perform morphological operations to enhance cell shapes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    opened = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel)

    # Find contours of cell objects
    contours, _ = cv2.findContours(opened, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area and circularity
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * (area / (perimeter ** 2))
        if area > 100 and circularity > 0.7:
            filtered_contours.append(contour)

    # Draw contours on the color image
    color_image = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    cv2.drawContours(color_image, filtered_contours, -1, (255, 0, 0), thickness=cv2.FILLED)

    return color_image


## 3.1 Experimental cell tracking

In [1]:
# Import libraries

import cv2
import numpy as np
import matplotlib.pyplot as plt

plt.switch_backend('Agg')


def preprocess_image(image):
    """
    Preprocess the image by converting to grayscale and applying Gaussian blur and thresholding.
    """
    if len(image.shape) == 3:  # Check if the image is color (3 channels)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:  # Grayscale image
        gray = image.copy()

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, threshold = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return threshold


def select_initial_objects(contours):
    """
    Select initial 5 objects based on area and circularity.
    """
    filtered_contours = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        circularity = 4 * np.pi * (area / (perimeter ** 2))
        if area > 100 and circularity > 0.7:
            filtered_contours.append(contour)
            if len(filtered_contours) == 5:
                break
    return filtered_contours

def track_objects(contours, tracked_objects):
    """
    Track previously identified objects based on last seen coordinates.
    """
    filtered_contours = []
    for cx, cy in tracked_objects:
        closest_contour = None
        closest_distance = float("inf")
        for contour in contours:
            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)
            circularity = 4 * np.pi * (area / (perimeter ** 2))
            if area > 100 and circularity > 0.7:
                moments = cv2.moments(contour)
                if moments["m00"] != 0:
                    new_cx = int(moments["m10"] / moments["m00"])
                    new_cy = int(moments["m01"] / moments["m00"])
                    distance = np.sqrt((cx - new_cx) ** 2 + (cy - new_cy) ** 2)
                    if distance < closest_distance:
                        closest_distance = distance
                        closest_contour = contour
        if closest_contour is not None:
            filtered_contours.append(closest_contour)
    return filtered_contours

def segment_images(images):
    tracked_objects = None
    thresholded_images = []

    for image in images:
        threshold = preprocess_image(image)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
        opened = cv2.morphologyEx(threshold, cv2.MORPH_OPEN, kernel)
        contours, _ = cv2.findContours(opened.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        filtered_contours = []
        if tracked_objects is None:
            filtered_contours = select_initial_objects(contours)
            tracked_objects = []
            for contour in filtered_contours:
                moments = cv2.moments(contour)
                if moments["m00"] != 0:
                    cx = int(moments["m10"] / moments["m00"])
                    cy = int(moments["m01"] / moments["m00"])
                    tracked_objects.append((cx, cy))

        else:
            filtered_contours = track_objects(contours, tracked_objects)

        color_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        for i, contour in enumerate(filtered_contours):
            moments = cv2.moments(contour)
            if moments["m00"] != 0:
                cx = int(moments["m10"] / moments["m00"])
                cy = int(moments["m01"] / moments["m00"])
                cv2.drawContours(color_image, [contour], -1, (255, 0, 0), thickness=cv2.FILLED)
                cv2.putText(color_image, text=str(i + 1), org=(cx, cy), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                            fontScale=1, color=(0, 0, 0), thickness=2, lineType=cv2.LINE_AA)


        thresholded_images.append(color_image)

    return thresholded_images

### Saving the images to folder

In [19]:
import os

# Load and process the images
images_ctrl = [np.mean(plt.imread(f"images/MTLn3-Ctrl00{'0' + str(i) if i < 10 else i}.tif"), -1).astype(np.uint8) for i in range(30)]

# Segment and track objects in the images
thresholded_images = segment_images(images_ctrl)

# Display the resulting images
for i, image in enumerate(thresholded_images):
    cv2.imshow(f"Image {i}", image)

    # if no ImTest folder, create one
    if not os.path.exists("ImTest"):
        os.makedirs("ImTest")


    # save the image to folder ImTest as png dpi 300
    cv2.imwrite(f"ImTest/Im{i}.png", image, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # cv2.waitKey(0)
    cv2.destroyAllWindows()

